# Imports e definição da interface gráfica

In [ ]:
from functools import partial
from typing import List, Tuple, Union

import numpy as np
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display

In [ ]:
X = +1
O = -1
EMPTY_SPACE = 0

INTERFACE_MAPPING = {
    X: 'X',
    O: 'O'
}

In [ ]:
class Interface:
    def __init__(self):
        self.buttons = [
            [
                widgets.Button(
                    description='',
                    layout=Layout(width='100px', height='100px')
                )
                for _ in range(3)
            ]
            for _ in range(3)
        ]

        self.board_widget = VBox([HBox(row) for row in self.buttons])

        self.__player = X
        self.link_positions()

    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the board

        :param pos: row, column of button on grid
        :param button: clicked button
        """
        if not button.description:
            button.description = INTERFACE_MAPPING.get(self.__player, '')
            self.__player *= -1  # invert player

    def link_positions(self):
        """Link clicks on buttons"""
        for i, row in enumerate(self.buttons):
            for j, button in enumerate(row):
                button.on_click(partial(self.on_select_position, (i, j)))

    def disable_buttons(self):
        """Disable buttons to avoid clicks after end game"""
        for row in self.buttons:
            for button in row:
                button.disabled = True


    def start(self):
        """Display board interface"""
        display(self.board_widget)

In [ ]:
interface = Interface()
interface.start()

In [ ]:
board = np.zeros((3, 3), dtype=int)
board[:, 1] = O
np.diag(board)
board

array([[ 0, -1,  0],
       [ 0, -1,  0],
       [ 0, -1,  0]])

# Sua vez, crie uma função para verificar se o jogo acabou, quais são as próximas possibilidades de jogadas e obter um score pro jogo

Pseudocode:

```python
receba uma board

somas_finalizadoras = [3, -3]

para cada linha:
    verifica se a soma é uma somas_finalizadoras

para cada coluna:
    verifica se a soma é uma somas_finalizadoras
    
verifica se a soma da diagonal principal é finalizadora

verifica se a soma da diagonal secundária é uma soma finalizadora

verifica se há posições vazias
```


In [ ]:
def get_game_status(board: np.ndarray) -> Tuple[bool, Union[int, None]]:
    """Check if game ended and who is the winner

    :param board:
    :return: if its a game over, who is the winner
    """
    # check if there is 3 of the same symbol in ...
    for player in [X, O]:
        # Check rows
        for row in board:
            if np.all(row == player):
                return True, player
        # Check columns
        for col in board.T:
            if np.all(col == player):
                return True, player
        # Check diagonals
        if np.all(np.diag(board) == player):
            return True, player
        if np.all(np.diag(np.fliplr(board)) == player):
            return True, player

    # Check for draw
    if not np.any(board == EMPTY_SPACE):
        return True, EMPTY_SPACE

    # Game not over
    return False, None

Pseudocode:

```python
receba uma board e player

possiveis_posicoes <- lista vazia
para cada posição da board:
    se vazio
        copia a board
        adiciona na posição o jogador
        adiciona no fim de possiveis_posicoes a copia

retorna possiveis_posicoes
```


In [ ]:
def get_possible_moves(board: np.ndarray, player: int = X) -> List[np.ndarray]:
    """Get next possible moves by some player
    """
    possible_positions = []
    for i in range(3):
        for j in range(3):
            if board[i, j] == EMPTY_SPACE:
                new_board = board.copy()
                new_board[i, j] = player
                possible_positions.append(new_board)
    return possible_positions

Pseudocode:

```python
receba um ganhador e a quantidade de movimentos

se o ganhador for a IA:
    retorne MAX_N_MOVES + 1 - n_movex
se for o humano
    retorne -(MAX_N_MOVES + 1 - n_movex)

senão
    retorna 0
```


In [ ]:
AI_PLAYER = X
HUMAN_PLAYER = O
MAX_N_MOVES = 9


def get_score(winner: int, n_moves: int) -> int:
    """Get how well was the game for the AI:
        - win faster is better than win slower
        - lose slower is better than win faster
        - draw is a intermediary result
    """
    if winner == AI_PLAYER:
        return MAX_N_MOVES + 1 - n_moves
    elif winner == HUMAN_PLAYER:
        return -(MAX_N_MOVES + 1 - n_moves)
    else:
        return 0

In [ ]:
# @markdown ## Testando fução de score

winner_str = "-1" # @param ["1", "-1", "0"] {allow-input: true}
winner = int(winner_str)
n_moves = 0 # @param {type:"slider", min:0, max:9, step:1}

print(get_score(winner, n_moves))

-10


# implemente o algorítimo de busca em profundidade MIni-max seguindo o pseudo código

Pseudocode:

```python
acabou, ganhador <- obtenha status do jogo
se o jogo acabou:
    returne o score do jogo e a board

proximo_jogador <- obtem próximo jogados

scores_e_movimentos <- lista vazia
para cada movimento possível:
    score, _ <- calcula o score e board utilizando mini_max
    addiciona o score e a boad ao fim da lista scores_e_movimentos

se o jogador for a AI_PLAYER:  # tentando maximizar
    returne o maximo de scores_e_movimentos

se não:
    returne o minimo de scores_e_movimentos
```


In [ ]:
def mini_max(board, player=AI_PLAYER, n_moves=0):
    # Get game status and if it over
    is_over, winner = get_game_status(board)
    # if game over return the score and board
    if is_over:
        return get_score(winner, n_moves), board

    # Get next possible moves
    possible_moves = get_possible_moves(board, player)

    scores_and_moves = []
    # For each possible move, call mini_max and get the score and board
    next_player = HUMAN_PLAYER if player == AI_PLAYER else AI_PLAYER
    for move in possible_moves:
        score, _ = mini_max(move, next_player, n_moves + 1)
        scores_and_moves.append((score, move))

    # choose the action to maximize the score if it is a maximization step
    if player == AI_PLAYER:  # trying to maximize
        return max(scores_and_moves, key=lambda x: x[0])
    # otherwise, choose the action to minimize the score
    else:
        return min(scores_and_moves, key=lambda x: x[0])

In [ ]:
board = np.zeros(shape=(3, 3), dtype=np.int8)
_, new_board = mini_max(board, AI_PLAYER)
new_board

array([[1, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int8)

In [ ]:
new_board = new_board.copy()
new_board[1, 0] = -1
_, new_board = mini_max(new_board, AI_PLAYER)
new_board

array([[ 1,  1,  0],
       [-1,  0,  0],
       [ 0,  0,  0]], dtype=int8)

# classe de interface

In [ ]:
END_GAME_MESSAGES = {
    AI_PLAYER: 'AI won!',
    HUMAN_PLAYER: 'You won!',
    EMPTY_SPACE: "It is a draw"
}


class TicTacToeAI(Interface):
    def __init__(self, ai_starts=False):
        super().__init__()
        self.board = np.zeros(shape=(3, 3), dtype=np.int8)
        if ai_starts:
            self.board = mini_max(self.board, AI_PLAYER)[1].copy()
            self.update()


    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the board
        It calls minmax algorithm after each user move

        :param pos: row, column of button on grid
        :param button: clicked button"""
        if not button.description:
            self.board[pos] = HUMAN_PLAYER
            self.board = mini_max(self.board, AI_PLAYER)[1].copy()

            is_over, winner = get_game_status(self.board)
            if is_over:
                self.disable_buttons()
                print(END_GAME_MESSAGES.get(winner))

            self.update()

    def update(self):
        """Update interface from virtual board"""
        for i, row in enumerate(self.board):
            for j, item in enumerate(row):
                self.buttons[i][j].description = INTERFACE_MAPPING.get(item, '')


# Hora de testar

In [ ]:
game = TicTacToeAI(ai_starts=False)
game.start()